In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)


# Dataset

In [4]:
df = pd.read_csv("../data/code-review-dataset-clear.csv")

In [5]:
X_train, X_val = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df["is_toxic"]
)

train_ds = Dataset.from_pandas(
    X_train[["message", "is_toxic"]].rename(columns={"message": "text", "is_toxic": "labels"}),
    preserve_index=False
)
val_ds = Dataset.from_pandas(
    X_val[["message", "is_toxic"]].rename(columns={"message": "text", "is_toxic": "labels"}),
    preserve_index=False
)

# RoBERTa


In [ ]:
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

train_ds = train_ds.map(tokenize_batch, batched=True)
val_ds   = val_ds.map(tokenize_batch, batched=True)

cols_to_keep = ["input_ids", "attention_mask", "labels"]
train_ds = train_ds.remove_columns([c for c in train_ds.column_names if c not in cols_to_keep])
val_ds   = val_ds.remove_columns([c for c in val_ds.column_names   if c not in cols_to_keep])


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "non_toxic", 1: "toxic"},
    label2id={"non_toxic": 0, "toxic": 1},
)


In [9]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="binary", pos_label=1, zero_division=0)
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}



In [ ]:
args = TrainingArguments(
    output_dir="./codeberta_toxic_cls",
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.238600,0.234445,0.904301,0.809639,0.666667,0.731230
2,0.219600,0.247551,0.913599,0.836930,0.692460,0.757872
3,0.139600,0.294675,0.907788,0.741818,0.809524,0.774194
4,0.120500,0.375381,0.913212,0.786885,0.761905,0.774194
5,0.163400,0.388431,0.911662,0.785124,0.753968,0.769231


TrainOutput(global_step=3230, training_loss=0.18459022281339663, metrics={'train_runtime': 233.4375, 'train_samples_per_second': 221.087, 'train_steps_per_second': 13.837, 'total_flos': 3394790391782400.0, 'train_loss': 0.18459022281339663, 'epoch': 5.0})

In [13]:
eval_metrics = trainer.evaluate()
print(eval_metrics)

{'eval_loss': 0.2946750819683075, 'eval_accuracy': 0.9077876791941109, 'eval_precision': 0.7418181818181818, 'eval_recall': 0.8095238095238095, 'eval_f1': 0.7741935483870968, 'eval_runtime': 1.7218, 'eval_samples_per_second': 1499.036, 'eval_steps_per_second': 47.045, 'epoch': 5.0}
